In [59]:
!pip install textract

In [73]:
from gensim.models import Word2Vec
import string
import re
import os
import collections
from os import listdir
from os.path import isfile, join
from io import StringIO
import pandas as pd
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()
from spacy.matcher import PhraseMatcher
from itertools import chain
import textract
from gensim.models import Word2Vec
import string
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import gensim
from gensim.models.phrases import Phraser, Phrases
import nltk
import collections
import PyPDF2
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [81]:
def Preprocessfile(filename):
  text = textract.process(filename)
  text= text.decode('utf-8').replace("\\n", " ")
  # print(text)
  x=[]
  tokens=word_tokenize(text)
  tok=[w.lower() for w in tokens]
  table=str.maketrans('','',string.punctuation)
  strpp=[w.translate(table) for w in tok]
  words=[word for word in strpp if word.isalpha()]
  stop_words=set(stopwords.words('english'))
  words=[w for w in words if not w in stop_words]
  x.append(words)
  # print(x)
  res=" ".join(chain.from_iterable(x))
  return res

In [148]:
x=Preprocessfile('jobtype.txt')
y=Preprocessfile('resume.pdf')
text=[y,x]



#print the similarity score
print("\n Similarity Score: ")
cv = CountVectorizer()
count_matrix = cv.fit_transform(text)
print(cosine_similarity(count_matrix))
matchpercent = cosine_similarity(count_matrix)[0][1]*100
matchpercent = round(matchpercent,2)

print("Your Resume matches about " + str(matchpercent) + "% of the job")


 Similarity Score: 
[[1.         0.51460293]
 [0.51460293 1.        ]]
Your Resume matches about 51.46% of the job


In [144]:
#all custom keywords should be in lower case
def find_score(jobdes,filename,setKeywords,customKeywords,df):   
    resume=Preprocessfile(filename)
    customKeywords = ' '.join(customKeywords)
    jobdes=jobdes + ' ' + customKeywords
    text=[resume,jobdes]
    cv = CountVectorizer()
    count_matrix = cv.fit_transform(text)
    print(cosine_similarity(count_matrix))
    matchpercent = cosine_similarity(count_matrix)[0][1]*100
    matchpercent = round(matchpercent,2)
    print(matchpercent)
    length=len(setKeywords)  #need to add more keywords and improve model
    Dict={
        'Data Science':'datascience',
        'Machine Learning':'machinelearning',
        'Web Development':'webdev',
        'Human resources':'hr',
        'App Development':'appdev'
    }
    keyword=[]
    for i in range(0,length):
      keyword.append([nlp(resume) for resume in df[Dict[setKeywords[i]]].dropna(axis = 0)])
    matcher = PhraseMatcher(nlp.vocab)
    for i in range(0,length) :
      matcher.add(Dict[setKeywords[i]],None,*keyword[i])
    doc = nlp(resume)
    matches = matcher(doc)
    print(matches)
    #The following is not required but additional data of the score can be obtained with the dataframe
    KEYS = []
    WORDS= []
    for match_id, start, end in matches:
        keys = nlp.vocab.strings[match_id]  
        words = doc[start : end]               
        print(f'{keys}  {words}')
        KEYS.append(keys);
        WORDS.append(words);
    DF=pd.DataFrame(KEYS,columns=['Type'])
    DF['Keyword']=WORDS
    print(DF)
    result={
        'score': matchpercent,
        'totalmatches':len(matches)
    }
    for i in range(0,length) :
      result[setKeywords[i]]=len(DF.loc[DF['Type']==Dict[setKeywords[i]]])
    
    return result

In [150]:
#FLOW
setKeywords=['Data Science','Machine Learning']
customKeywords=['spanish','hindi','opencv']


jobdes=Preprocessfile('jobtype.txt')  #the job description is preprocessed outside as the same job description is used for the multiple resumes

df = pd.read_csv('setkeywords.csv') #all the processed similar keywords are stored in a smaller csv file instead of a .model file

#iterate through all resumes here
results=find_score(jobdes,'resume.pdf',setKeywords,customKeywords,df);



[[1.         0.51409916]
 [0.51409916 1.        ]]
51.41
[(14878926805768099270, 0, 1), (14878926805768099270, 17, 18), (14878926805768099270, 27, 28), (14878926805768099270, 27, 29), (14878926805768099270, 53, 54), (14878926805768099270, 53, 55), (9223223603407622304, 55, 57), (14878926805768099270, 55, 57), (14878926805768099270, 58, 59), (14878926805768099270, 60, 61), (14878926805768099270, 60, 62), (9223223603407622304, 62, 64), (14878926805768099270, 62, 64), (9223223603407622304, 64, 65), (14878926805768099270, 64, 65), (14878926805768099270, 65, 66), (9223223603407622304, 66, 67), (14878926805768099270, 66, 67), (14878926805768099270, 76, 77), (14878926805768099270, 148, 149), (14878926805768099270, 157, 158), (9223223603407622304, 159, 161), (14878926805768099270, 159, 161), (14878926805768099270, 175, 176), (14878926805768099270, 190, 191), (14878926805768099270, 192, 193), (14878926805768099270, 192, 194), (9223223603407622304, 195, 196), (14878926805768099270, 195, 196), (1

In [151]:
results #each number of matches is returned.

{'Data Science': 24, 'Machine Learning': 6, 'score': 51.41, 'totalmatches': 30}